<figure>
  <IMG SRC="https://raw.githubusercontent.com/fmeer/public-files/main/TUlogo.png" WIDTH=200 ALIGN="right">
</figure>

# CIEM5110-2 Workshop 4.2: Nonlinear FEM with plastic hinges and arclength (notebook 3)

## Frame with lateral loading
We exame the same frame with a different loading. This is the load case that has been discussed in the plasticity lectures of *Stability of Structures*. We use $L=2$, $EI=10$, $EA=20000$, $GA=10000$.

<center><img src="https://raw.githubusercontent.com/fmeer/public-files/main/ciem5110/frameB.png" alt="Simple frame with lateral load" width="300"/></center>



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../../')

from utils import proputils as pu
import main
from names import GlobNames as gn

%matplotlib tk

### Geometrically nonlinear analysis

Starting point is geometrically nonlinear analysis, with input as in workshop 4.1. 

In [ ]:
plt.close('all')
props = pu.parse_file('nonlinear.pro')
globdat = main.jive(props)

def getFu(globdat):
    F = globdat['loaddisp']['topleft']['load']['dx']
    u = globdat['loaddisp']['topleft']['disp']['dx']
    return np.vstack((u,F))

FuPlasNL = getFu(globdat)

To inspect some more results, we can create a new `FrameViewModule`.

In [ ]:


fv = globdat[gn.MODULEFACTORY].get_module('FrameView','fv')

props['fv'] = {}
props['fv']['plotStress'] = 'M'
props['fv']['deform'] = '1'
props['fv']['interactive'] = 'True'
props['fv']['plotNeumann'] = 'False'
props['fv']['step0'] = 100

plt.close('all')
fv.init(props, globdat)
status = fv.run(globdat)
fv.shutdown(globdat)



### Geometrically linear version

We add plastic hinges to the analysis with two additional properties in the input file for nonlinear analysis. From the immediate convergence and from the shape of the load-displacement curve, it can be observed that the analysis is indeed linear (except for discrete events when the plastic hinges are added). Do you notice the difference in displacements compared to the nonlinear analysis?

In [ ]:
plt.close('all')
props = pu.parse_file('nonlinear.pro')
props['model']['frame']['subtype'] = 'linear';
globdat = main.jive(props)

FuPlasLin = getFu(globdat)

In [ ]:
Mp = float(props['model']['frame']['Mp'])
L = 2
uanalytical = np.linspace(0,0.3,100)
Fanalytical = 3*Mp/L*(1-43/3*uanalytical/L)

In [ ]:
plt.close('all')
plt.plot(FuPlasNL[0],FuPlasNL[1])
plt.plot(FuPlasLin[0],FuPlasLin[1])
plt.plot(uanalytical,Fanalytical,'--')
plt.plot(uanalytical,Fanalytical[0]*np.ones(uanalytical.shape),'--')
plt.ylim(0,0.7)
plt.show()

It is a bit hard to judge the level of agreement between rigid-plastic and nonlinear FE solution. This is due to the fact that displacements are already significant when the mechanism develops in the FE simulation. Therefore, the second order approximation in the rigid-plastic solution is not realistic in the region where the two can be compared. 

To check the analytical result, we can let the FE solution behave more in agreement with the rigid-plastic assumptions. This is done by increasing the stiffnesses.

In [ ]:
plt.close('all')
props = pu.parse_file('nonlinear.pro')
props['model']['frame']['EI'] = '2.e3'
props['model']['frame']['GA'] = '1.e6'
props['model']['frame']['EA'] = '2.e6'
globdat = main.jive(props)

def getFu(globdat):
    F = globdat['loaddisp']['topleft']['load']['dx']
    u = globdat['loaddisp']['topleft']['disp']['dx']
    return np.vstack((u,F))

FuRigPlas = getFu(globdat)

In [ ]:
plt.close('all')
plt.plot(FuPlasNL[0],FuPlasNL[1],label='nonlinear elastic/plastic')
plt.plot(FuPlasLin[0],FuPlasLin[1],label='linear elastic/plastic')
plt.plot(FuRigPlas[0],FuRigPlas[1],label='nonlinear rigid/plastic')
plt.plot(uanalytical,Fanalytical,'--',label='analytical 2nd order')
plt.plot(uanalytical,Fanalytical[0]*np.ones(uanalytical.shape),'--',label='analytical 1st order')
plt.xlabel('u')
plt.ylabel('F')
plt.ylim(0,0.7)
plt.legend()
plt.show()

Now, we can see that the analytical second order approximation indeed gives an accurate representation of the initial slope of the force-displacement relation in the plastic mechanism.

The comparison above indicates that the buckling load is of the same order of magnitude as the plastic collapse load, because the maximum load from complete nonlinear analysis including both plasticity and geometric nonlinearity is significantly lower than the plastic collapse load from linear analysis. With linear buckling analysis we can assert whether this is indeed the case. We remove the `ArclenModule` from the `props` and add a `LinBuckModule`

In [ ]:
plt.close('all')
props = pu.parse_file('nonlinear.pro')
del props['nonlin']
del props['loaddisp']
del props['graph']

props['model']['neum']['values'] = props['model']['neum']['loadIncr']
props['linbuck'] = {}
props['linbuck']['type'] = 'LinBuck'
globdat = main.jive(props)


Finally, we can check the accuracy of Merchant's formula for predicting the maximum load $F_\mathrm{max}$ from the linear plastic collapse load $F_\mathrm{p}$ and the linear buckling load $F_\mathrm{b}$ through
$$\frac{1}{F_\mathrm{max}} = \frac{1}{F_\mathrm{p}} + \frac{1}{F_\mathrm{b}}$$

In [ ]:
Fp = Fanalytical[0]
Fb = globdat[gn.LBFACTORS][0].real
Fmerchant = 1/(1/Fp+1/Fb)
FmaxFEM = max(FuPlasNL[1])
print('Fp        = ' + str(Fp))
print('Fb        = ' + str(Fb))
print('Fmerchant = ' + str(Fmerchant))
print('FmaxFEM   = ' + str(FmaxFEM))